# README - Collecte et Traitement des Données ERA5 pour Ålesund

Ce notebook détaille le processus de collecte, de nettoyage et de sauvegarde de données météorologiques issues de la réanalyse ERA5 (Copernicus Climate Data Store) pour la région d'Ålesund.

## Étapes principales

1. **Configuration de l'accès aux données ERA5**
    - Création du fichier `.cdsapirc` avec vos identifiants CDS API.
    - Installation des packages nécessaires : `cdsapi`, `xarray[io]`, `pandas`, `numpy`.

2. **Téléchargement des variables climatiques**
    - Variables traitées : pression de surface, rayonnement solaire, vent (composantes U et V à 10m), température à 2m.
    - Téléchargement automatisé pour les années 2021 à 2025 sur la zone d'intérêt.

3. **Conversion et sauvegarde**
    - Conversion des fichiers NetCDF en DataFrame puis en CSV pour chaque variable.

4. **Nettoyage des données**
    - Conversion des dates.
    - Détection et suppression des doublons et des valeurs manquantes.
    - Sauvegarde des fichiers nettoyés.

## Structure des fichiers

- Les fichiers CSV générés sont nommés selon la variable et la zone (ex : `u10_alesund.csv`, `2m_temperature_alesund.csv`).
- Les fichiers NetCDF intermédiaires sont supprimés après concaténation.

## Remarques

- Les unités des variables sont précisées dans les cellules markdown du notebook.
- Les scripts sont adaptés pour fonctionner sur un environnement local Windows, mais peuvent être adaptés à d'autres systèmes.
- Pour toute utilisation, veillez à respecter les conditions d'utilisation des données ERA5.

## Références

- [ERA5 - Copernicus Climate Data Store](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=documentation)
- [Documentation xarray](https://docs.xarray.dev/)
- [Documentation pandas](https://pandas.pydata.org/)

In [ ]:
import pandas as pd # Importing necessary libraries for data manipulation
import numpy as np  # Importing numpy for numerical operations
import cdsapi       # Importing cdsapi for accessing Climate Data Store API
import os          # Importing os for operating system dependent functionality
import xarray as xr        # Importing xarray for working with labeled multi-dimensional arrays

collect des données d'après le site : https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=documentation (ERA5)

pour lire les fichier format  NetCDF j'ai instaler sur mon env virtual le package pip install xarray[io]


avant d'execute le code Créer le fichier de configuration

Créez le fichier .cdsapirc dans votre répertoire utilisateur :

Chemin : C:\Users\nom\.cdsapirc


Contenu du fichier  :  

url: https://cds.climate.copernicus.eu/api/v2

key: UID:API_KEY
#### Où trouver le UID et l'API Key ?

Connecte-toi sur le site :
 https://cds.climate.copernicus.eu

Clique sur "My Account" (ou "Mon compte")

Tu verras un encadré avec :

    UID: 123***

    API Key (API Token): abcde-...

Copie ces deux éléments et colle-les dans le fichier .cdsapirc


In [ ]:
# Initialisation du client CDS API
client = cdsapi.Client()

variables = ["surface_pressure",
"surface_solar_radiation_downwards",
"10m_u_component_of_wind",
"10m_v_component_of_wind",
"2m_temperature"

]

years = ["2021", "2022", "2023", "2024", "2025"]  # Liste des années à traiter

# Dossiers de travail
temp_dir = "temp_nc"
output_dir = "data_era5"
os.makedirs(temp_dir, exist_ok=True)   # Créer le dossier temporaire pour les fichiers NetCDF
os.makedirs(output_dir, exist_ok=True)   # Créer le dossier de sortie pour les fichiers finaux

# Configuration commune
common_request = {
    "product_type": "reanalysis",
    "month": [f"{m:02d}" for m in range(1, 13)],
    "day": [f"{d:02d}" for d in range(1, 32)],
    "time": [f"{h:02d}:00" for h in range(24)],
    "data_format": "netcdf",
    "area": [63, 6, 62, 6.3]  # Nord, Ouest, Sud, Est
}

for var in variables:
    yearly_files = []

    for year in years:
        request = common_request.copy()
        request["variable"] = var
        request["year"] = year

        temp_file = os.path.join(temp_dir, f"{var}_{year}.nc")
        print(f" Téléchargement {var} {year}...")

        if not os.path.exists(temp_file):
            try:
                client.retrieve(
                    "reanalysis-era5-single-levels",
                    request
                ).download(temp_file)
            except Exception as e:
                print(f" Erreur pour {var} {year} : {e}")
                continue

        yearly_files.append(temp_file)

    # Concaténer les fichiers avec engine="h5netcdf"
    print(f" Fusion des fichiers pour {var}...")
   # Ouverture + concaténation des fichiers
datasets = [xr.open_dataset(f, engine="h5netcdf") for f in yearly_files]
combined = xr.concat(datasets, dim="time")


# Sauvegarde
final_path = os.path.join(output_dir, f"{var}_2021_2025.nc")

combined.to_netcdf(final_path, engine="h5netcdf")
print(f" Fichier final : {final_path}")

# Nettoyage
for ds in datasets:
    ds.close()
for f in yearly_files:
    os.remove(f)



2025-05-28 15:27:54,976 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


 Téléchargement 2m_temperature 2021...
 Téléchargement 2m_temperature 2022...
 Téléchargement 2m_temperature 2023...
 Téléchargement 2m_temperature 2024...
 Téléchargement 2m_temperature 2025...
 Fusion des fichiers pour 2m_temperature...
 Fichier final : data_era5\2m_temperature_2021_2025.nc


In [ ]:

# Ouvrir le fichier NetCDF
ds = xr.open_dataset("10m_u_component_of_wind_2021_2025.nc", engine="h5netcdf")

# Convertir en DataFrame (aplatir toutes les dimensions)
df = ds.to_dataframe().reset_index()

# Sauvegarder au format CSV
df.to_csv("u10_alesund.csv", index=False)

print("Conversion terminée : fichier CSV créé.")


Conversion terminée : fichier CSV créé.


In [11]:
df_wind_u10=pd.read_csv('u10_alesund.csv')
df_wind_u10

,Unnamed: 0,time,valid_time,latitude,longitude,u10,number,expver
0,8,0,2021-01-01 00:00:00,62.00,6.00,-0.330505,0,1.0
1,9,0,2021-01-01 00:00:00,62.00,6.25,0.145081,0,1.0
2,6,0,2021-01-01 00:00:00,62.25,6.00,-2.230896,0,1.0
3,7,0,2021-01-01 00:00:00,62.25,6.25,-1.430115,0,1.0
4,4,0,2021-01-01 00:00:00,62.50,6.00,-3.520935,0,1.0
...,...,...,...,...,...,...,...,...
384615,1923095,4,2025-05-22 13:00:00,62.50,6.25,-3.932678,0,5.0
384616,1923092,4,2025-05-22 13:00:00,62.75,6.00,-6.191467,0,5.0
384617,1923093,4,2025-05-22 13:00:00,62.75,6.25,-5.601623,0,5.0
384618,1923090,4,2025-05-22 13:00:00,63.00,6.00,-6.256897,0,5.0


In [12]:
df_wind_u10.loc[:, 'valid_time'] = pd.to_datetime(df_wind_u10.loc[:, 'valid_time'])
nan_par_colonne = df_wind_u10.isna().sum()
print(nan_par_colonne)

Unnamed: 0    0
time          0
valid_time    0
latitude      0
longitude     0
u10           0
number        0
expver        0
dtype: int64


In [13]:
nbr_doublons = df_wind_u10.duplicated(subset=['valid_time', 'latitude', 'longitude']).sum()
print("Nombre de doublons sur ['valid_time', 'latitude', 'longitude'] :", nbr_doublons)


Nombre de doublons sur ['valid_time', 'latitude', 'longitude'] : 0


In [7]:
# Créer une colonne qui indique si la ligne contient au moins un NaN
df_wind_u10['has_nan'] = df_wind_u10.isna().any(axis=1)

# Trier pour que les lignes sans NaN viennent avant celles avec NaN,
# puis trier par les colonnes clés pour grouper les doublons
df_wind_u10 = df_wind_u10.sort_values(by=['valid_time', 'latitude', 'longitude', 'has_nan'])

# Supprimer les doublons sur les colonnes clés, garder la première occurrence (celle sans NaN si elle existe)
df_wind_u10 = df_wind_u10.drop_duplicates(subset=['valid_time', 'latitude', 'longitude'], keep='first')

# Supprimer la colonne auxiliaire
df_wind_u10 = df_wind_u10.drop(columns='has_nan')


In [10]:
df_wind_u10.to_csv('u10_alesund.csv')

Ce paramètre représente la composante nord du vent à 10 m. Il s'agit de la vitesse horizontale de l'air se déplaçant vers le nord, à une altitude de dix mètres au-dessus de la surface de la Terre, en mètres par seconde. Il convient d'être prudent lors de la comparaison de ce paramètre avec les observations, car celles-ci varient sur de courtes échelles spatiales et temporelles et sont influencées par le relief, la végétation et les bâtiments locaux, qui ne sont représentés qu'en moyenne dans le Système intégré de prévision du CEPMMT.

Ce paramètre peut être combiné à la composante U du vent à 10 m pour donner la vitesse et la direction du vent horizontal à 10 m.

In [14]:

# Ouvrir le fichier NetCDF
ds = xr.open_dataset(r"C:\Users\emac\OneDrive - IMT Mines Albi\Projet stage\Data de projet ORION\notebook collect des données\data_era5\10m_v_component_of_wind_2021_2025.nc", engine="h5netcdf")

# Convertir en DataFrame (aplatir toutes les dimensions)
df = ds.to_dataframe().reset_index()

# Sauvegarder au format CSV
df.to_csv("v10_alesund.csv", index=False)

print("Conversion terminée : fichier CSV créé.")

Conversion terminée : fichier CSV créé.


In [22]:
df_wind_V10=pd.read_csv('v10_alesund.csv')
df_wind_V10

,Unnamed: 0,time,valid_time,latitude,longitude,v10,number,expver
0,8,0,2021-01-01 00:00:00,62.00,6.00,-1.195236,0,1.0
1,9,0,2021-01-01 00:00:00,62.00,6.25,-1.080002,0,1.0
2,6,0,2021-01-01 00:00:00,62.25,6.00,-1.651291,0,1.0
3,7,0,2021-01-01 00:00:00,62.25,6.25,-1.052658,0,1.0
4,4,0,2021-01-01 00:00:00,62.50,6.00,-1.738205,0,1.0
...,...,...,...,...,...,...,...,...
384625,1923145,4,2025-05-22 14:00:00,62.50,6.25,-6.744537,0,5.0
384626,1923142,4,2025-05-22 14:00:00,62.75,6.00,-8.470123,0,5.0
384627,1923143,4,2025-05-22 14:00:00,62.75,6.25,-7.762116,0,5.0
384628,1923140,4,2025-05-22 14:00:00,63.00,6.00,-8.040436,0,5.0


In [23]:
df_wind_V10.loc[:, 'valid_time'] = pd.to_datetime(df_wind_V10.loc[:, 'valid_time'])
nan_par_colonne = df_wind_V10.isna().sum()
print(nan_par_colonne)

Unnamed: 0    0
time          0
valid_time    0
latitude      0
longitude     0
v10           0
number        0
expver        0
dtype: int64


In [24]:
nbr_doublons = df_wind_V10.duplicated(subset=['valid_time', 'latitude', 'longitude']).sum()
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [18]:
# Créer une colonne qui indique si la ligne contient au moins un NaN
df_wind_V10['has_nan'] = df_wind_V10.isna().any(axis=1)

# Trier pour que les lignes sans NaN viennent avant celles avec NaN,
# puis trier par les colonnes clés pour grouper les doublons
df_wind_V10 = df_wind_V10.sort_values(by=['valid_time', 'latitude', 'longitude', 'has_nan'])

# Supprimer les doublons sur les colonnes clés, garder la première occurrence (celle sans NaN si elle existe)
df_wind_V10 = df_wind_V10.drop_duplicates(subset=['valid_time', 'latitude', 'longitude'], keep='first')

# Supprimer la colonne auxiliaire
df_wind_V10 = df_wind_V10.drop(columns='has_nan')


In [21]:
df_wind_V10.to_csv('v10_alesund.csv')

Ce paramètre correspond à la température de l'air à 2 m au-dessus de la surface terrestre, de la mer ou des eaux intérieures.

La température à 2 m est calculée par interpolation entre le niveau le plus bas du modèle et la surface terrestre, en tenant compte des conditions atmosphériques.

Ce paramètre est exprimé en kelvins (K). La température mesurée en kelvins peut être convertie en degrés Celsius (°C) en soustrayant 273,15.



In [2]:
# Ouvrir le fichier NetCDF
ds = xr.open_dataset(r"C:\Users\emac\OneDrive - IMT Mines Albi\Projet stage\Data de projet ORION\notebook collect des données\data_era5\2m_temperature_2021_2025.nc", engine="h5netcdf")

# Convertir en DataFrame (aplatir toutes les dimensions)
df = ds.to_dataframe().reset_index()

# Sauvegarder au format CSV
df.to_csv("2m_temperature_alesund.csv", index=False)

print("Conversion terminée : fichier CSV créé.")

Conversion terminée : fichier CSV créé.


Unit K

In [13]:
df_temp_alesund=pd.read_csv('2m_temperature_alesund.csv')
df_temp_alesund

,Unnamed: 0,time,valid_time,latitude,longitude,t2m,number,expver
0,0,0,2021-01-01 00:00:00,63.00,6.00,278.54040,0,1.0
1,1,0,2021-01-01 00:00:00,63.00,6.25,278.28650,0,1.0
2,2,0,2021-01-01 00:00:00,62.75,6.00,278.49940,0,1.0
3,3,0,2021-01-01 00:00:00,62.75,6.25,277.45837,0,1.0
4,4,0,2021-01-01 00:00:00,62.50,6.00,276.17517,0,1.0
...,...,...,...,...,...,...,...,...
384855,1924295,4,2025-05-23 13:00:00,62.50,6.25,280.78406,0,5.0
384856,1924296,4,2025-05-23 13:00:00,62.25,6.00,280.98523,0,5.0
384857,1924297,4,2025-05-23 13:00:00,62.25,6.25,281.23523,0,5.0
384858,1924298,4,2025-05-23 13:00:00,62.00,6.00,281.70007,0,5.0


In [10]:
df_temp_alesund.loc[:, 'valid_time'] = pd.to_datetime(df_temp_alesund.loc[:, 'valid_time'])
nan_par_colonne = df_temp_alesund.isna().sum()
print(nan_par_colonne)

time          0
valid_time    0
latitude      0
longitude     0
t2m           0
number        0
expver        0
dtype: int64


In [11]:
nbr_doublons = df_temp_alesund.duplicated(subset=['valid_time', 'latitude', 'longitude']).sum()
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [9]:
# Créer une colonne indiquant si une ligne contient un NaN
df_temp_alesund['has_nan'] = df_temp_alesund.isna().any(axis=1)

# Trier les lignes pour que celles sans NaN viennent en premier
df_temp_alesund = df_temp_alesund.sort_values(by=['valid_time', 'has_nan'])

# Supprimer les doublons en gardant la première (celle sans NaN si elle existe)
df_temp_alesund = df_temp_alesund.drop_duplicates(subset=['valid_time', 'latitude', 'longitude'], keep='first')

# Supprimer la colonne auxiliaire
df_temp_alesund = df_temp_alesund.drop(columns='has_nan')

In [12]:
df_temp_alesund.to_csv('2m_temperature_alesund.csv')

$J \, m^{-2}$


Ce paramètre représente la quantité de rayonnement solaire (également appelé rayonnement à ondes courtes) qui atteint un plan horizontal à la surface de la Terre. Ce paramètre comprend le rayonnement solaire direct et diffus.

Le rayonnement solaire (rayonnement solaire, ou à ondes courtes) est en partie réfléchi vers l'espace par les nuages ​​et les particules de l'atmosphère (aérosols) et une partie est absorbée. Le reste est incident à la surface de la Terre (représenté par ce paramètre)
Ce paramètre représente la quantité de rayonnement solaire (également appelé rayonnement à ondes courtes) qui atteint un plan horizontal à la surface de la Terre. Ce paramètre comprend le rayonnement solaire direct et diffus.

Le rayonnement solaire (rayonnement solaire, ou à ondes courtes) est en partie réfléchi vers l'espace par les nuages ​​et les particules de l'atmosphère (aérosols) et une partie est absorbée. Le reste est incident à la surface de la Terre (représenté par ce paramètre).


In [2]:
# Ouvrir le fichier NetCDF
ds = xr.open_dataset(r"C:\Users\emac\OneDrive - IMT Mines Albi\Projet stage\Data de projet ORION\notebook collect des données\data_era5\surface_solar_radiation_downwards_2021_2025.nc", engine="h5netcdf")

# Convertir en DataFrame (aplatir toutes les dimensions)
df = ds.to_dataframe().reset_index()

# Sauvegarder au format CSV
df.to_csv("surface_solar_radiation_downwards_alesund.csv", index=False)

print("Conversion terminée : fichier CSV créé.")

Conversion terminée : fichier CSV créé.


In [3]:
df_radiation_alesund=pd.read_csv('surface_solar_radiation_downwards_alesund.csv')
df_radiation_alesund

,time,valid_time,latitude,longitude,ssrd,number,expver
0,0,2021-01-01 00:00:00,63.00,6.00,0.0,0,1.0
1,0,2021-01-01 00:00:00,63.00,6.25,0.0,0,1.0
2,0,2021-01-01 00:00:00,62.75,6.00,0.0,0,1.0
3,0,2021-01-01 00:00:00,62.75,6.25,0.0,0,1.0
4,0,2021-01-01 00:00:00,62.50,6.00,0.0,0,1.0
...,...,...,...,...,...,...,...
1924045,4,2025-05-23 08:00:00,62.50,6.25,1177216.0,0,5.0
1924046,4,2025-05-23 08:00:00,62.25,6.00,1184256.0,0,5.0
1924047,4,2025-05-23 08:00:00,62.25,6.25,1148672.0,0,5.0
1924048,4,2025-05-23 08:00:00,62.00,6.00,1250304.0,0,5.0


In [8]:
df_radiation_alesund.loc[:, 'valid_time'] = pd.to_datetime(df_radiation_alesund.loc[:, 'valid_time'])
nan_par_colonne = df_radiation_alesund.isna().sum()
print(nan_par_colonne)

time          0
valid_time    0
latitude      0
longitude     0
ssrd          0
number        0
expver        0
dtype: int64


In [7]:
nbr_doublons = df_radiation_alesund.duplicated(subset=['valid_time', 'latitude', 'longitude']).sum()
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [6]:
# Créer une colonne indiquant si une ligne contient un NaN
df_radiation_alesund['has_nan'] = df_radiation_alesund.isna().any(axis=1)

# Trier les lignes pour que celles sans NaN viennent en premier
df_radiation_alesund = df_radiation_alesund.sort_values(by=['valid_time', 'has_nan'])

# Supprimer les doublons en gardant la première (celle sans NaN si elle existe)
df_radiation_alesund = df_radiation_alesund.drop_duplicates(subset=['valid_time','latitude','longitude'], keep='first')

# Supprimer la colonne auxiliaire
df_radiation_alesund = df_radiation_alesund.drop(columns='has_nan')

In [10]:
df_radiation_alesund.to_csv('surface_solar_radiation_downwards_alesund.csv')

Ce paramètre représente la pression (force par unité de surface) de l'atmosphère à la surface des terres, des mers et des eaux intérieures.

Il mesure le poids de l'air dans une colonne verticalement au-dessus de la surface terrestre, représentée en un point fixe.

La pression de surface est souvent utilisée en combinaison avec la température pour calculer la densité de l'air.

La forte variation de pression avec l'altitude rend difficile la visualisation des systèmes de basse et haute pression au-dessus des zones montagneuses ; c'est donc la pression moyenne au niveau de la mer, plutôt que la pression de surface, qui est généralement utilisée à cette fin.

L'unité de ce paramètre est le pascal (Pa). La pression de surface est souvent mesurée en hPa et parfois exprimée dans l'ancienne unité, le millibar (mb) (1 hPa = 1 mb = 100 Pa).

In [30]:
# Ouvrir le fichier NetCDF
ds = xr.open_dataset(r"C:\Users\emac\OneDrive - IMT Mines Albi\Projet stage\Data de projet ORION\notebook collect des données\data_era5\surface_pressure_2021_2025.nc", engine="h5netcdf")

# Convertir en DataFrame (aplatir toutes les dimensions)
df = ds.to_dataframe().reset_index()

# Sauvegarder au format CSV
df.to_csv("surface_pressure.csv", index=False)

print("Conversion terminée : fichier CSV créé.")

Conversion terminée : fichier CSV créé.


In [37]:
df_surface_pressure=pd.read_csv('surface_pressure.csv')
df_surface_pressure

,Unnamed: 0,time,valid_time,latitude,longitude,sp,number,expver
0,0,0,2021-01-01 00:00:00,63.00,6.00,100588.56,0,1.0
1,1,0,2021-01-01 00:00:00,63.00,6.25,100655.56,0,1.0
2,2,0,2021-01-01 00:00:00,62.75,6.00,100801.56,0,1.0
3,3,0,2021-01-01 00:00:00,62.75,6.25,100350.56,0,1.0
4,4,0,2021-01-01 00:00:00,62.50,6.00,99503.56,0,1.0
...,...,...,...,...,...,...,...,...
384795,1923995,4,2025-05-23 07:00:00,62.50,6.25,99726.56,0,5.0
384796,1923996,4,2025-05-23 07:00:00,62.25,6.00,98042.56,0,5.0
384797,1923997,4,2025-05-23 07:00:00,62.25,6.25,96969.56,0,5.0
384798,1923998,4,2025-05-23 07:00:00,62.00,6.00,96490.56,0,5.0


In [38]:
nbr_doublons = df_surface_pressure.duplicated(subset=['valid_time', 'latitude', 'longitude']).sum()
print("Nombre de doublons dans startTime :", nbr_doublons)

Nombre de doublons dans startTime : 0


In [33]:
# Créer une colonne indiquant si une ligne contient un NaN
df_surface_pressure['has_nan'] = df_surface_pressure.isna().any(axis=1)

# Trier les lignes pour que celles sans NaN viennent en premier
df_surface_pressure = df_surface_pressure.sort_values(by=['valid_time', 'has_nan'])

# Supprimer les doublons en gardant la première (celle sans NaN si elle existe)
df_surface_pressure = df_surface_pressure.drop_duplicates(subset=['valid_time','latitude', 'longitude'], keep='first')

# Supprimer la colonne auxiliaire
df_surface_pressure = df_surface_pressure.drop(columns='has_nan')

In [39]:
df_surface_pressure.loc[:, 'valid_time'] = pd.to_datetime(df_surface_pressure.loc[:, 'valid_time'])
nan_par_colonne = df_surface_pressure.isna().sum()
print(nan_par_colonne)

Unnamed: 0    0
time          0
valid_time    0
latitude      0
longitude     0
sp            0
number        0
expver        0
dtype: int64


In [36]:
df_surface_pressure.to_csv('surface_pressure.csv')